In [8]:
import os.path
import string
from os import listdir
import numpy as np
import pandas as pd
import pickle


#Folder containing the labeled snippet subfolders for training the classifier
#snippetFolder = "/home/abc/ADM/Train_Data" #for Chandra
snippetFolder = "Train_Data" #for Dan

#folder containing unlabeled snippets
#testSnippetFolder = "/home/abc/ADM/Test_Data" #for Chandra
testSnippetFolder = "Test_Data" #for Dan

#folderToSaveResultsFile = "/home/abc/ADM/Results" #for Chandra
folderToSaveResultsFile = "Results" #for Dan

#list of all folders in the training data
folder_list = [f for f in listdir(snippetFolder) if not f.startswith('.')]

puncs = string.punctuation

X = []
Y = []
for folder in folder_list:
    files_list = [f for f in listdir(snippetFolder + "/" + folder) if not f.startswith('.')]
    for file in files_list:
        filePath = snippetFolder + "/" + folder + "/" + file
        f = open(filePath,'r',encoding='utf-8')
        data = f.readlines()
        textSnippet = ''.join(data)
        textSnippet = textSnippet.replace("\n","")
        for p in puncs:
            if p in textSnippet:
                textSnippet = textSnippet.replace(p," ")
        X.append(textSnippet)
        Y.append(folder)
    
print(len(X))
print(len(Y))

#maximum number of features you want to consider to build the model
max_features = None #None means no limit

#which n-gram model to use? if low is 2 and high is 5, we will consider all 2-grams to 5-grams
low = 2 
high = 2

#word occurences
min_freq = 6 #np.ceil(len(X) / 10)
max_freq = np.ceil(len(X) / 1.5)

print(min_freq)
print(max_freq)

#number of folds for cross-validation
cv=5



#from sklearn.feature_extraction import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score

"""
tfidf = TfidfVectorizer(input='content',encoding='utf-8',decode_error='ignore',strip_accents='ascii',\
                        ngram_range=(2,2),stop_words='english',max_df=80,min_df=6,max_features=None,\
                        norm='l1')
"""
tfidf = TfidfVectorizer(input='content', encoding='utf-8', decode_error='ignore', strip_accents='ascii',\
                        ngram_range=(low,high), stop_words='english', max_features=max_features, norm='l1',\
                        max_df=max_freq, min_df=min_freq)
train = tfidf.fit_transform(X,Y)
print(len(tfidf.get_feature_names()))
print(tfidf.get_feature_names())
print(train.shape)
#print("\nWords Removed : \n",tfidf.stop_words_)
#print(tfidf.build_analyzer())


164
164
6
110.0
19
['architecture chicago', 'architecture history', 'armour institute', 'art institute', 'art school', 'building chicago', 'carl condit', 'chicago school', 'chicago university', 'civics philanthropy', 'condit chicago', 'institute armour', 'institute technology', 'new york', 'school architecture', 'school chicago', 'school civics', 'university chicago', 'university illinois']
(164, 19)


In [2]:
#DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

criterion = ["gini","entropy"]
splitter = ["best", "random"]


clf_dt = DecisionTreeClassifier(criterion="gini", splitter="best", max_features=None, max_depth=None, \
                                min_samples_split=2, min_samples_leaf=1, max_leaf_nodes=None, \
                                class_weight="balanced", random_state=123, presort=False)

dt_acc = cross_val_score(clf_dt,train,Y,scoring="accuracy",cv=3)
#print(acc)
print(np.mean(dt_acc))

clf_dt.fit(train, Y)

#print(len(clf_dt.feature_importances_))
#print(clf_dt.feature_importances_)
#print(clf_dt.tree_)
#tree.export_graphviz(clf_dt,out_file='tree.dot')
df = pd.DataFrame(data= {"Features" : tfidf.get_feature_names(), "weights" : clf_dt.feature_importances_})
df = df.sort_values(axis=0,by='weights',ascending=False)
print(df)


print(clf_dt.classes_)

"""
import io
import pydot

def show_tree(decisionTree, file_path):
    dotfile = io.StringIO()
    tree.export_graphviz(decisionTree, out_file=dotfile)
    pydot.graph_from_dot_data(dotfile.getvalue()).write_png(file_path)
    i = misc.imread(file_path)
    plt.imshow(i)

# To use it
show_tree(clf_dt, 'test.png')
"""
print(clf_dt.max_features_)
print(clf_dt.n_classes_)
print(clf_dt.n_features_)
print(clf_dt.n_outputs_)

0.531746031746
                Features   weights
16         school civics  0.431223
14   school architecture  0.177500
10        condit chicago  0.112025
7         chicago school  0.109621
17    university chicago  0.059994
3          art institute  0.047796
6            carl condit  0.021228
0   architecture chicago  0.016355
13              new york  0.013130
2       armour institute  0.011128
12  institute technology  0.000000
15        school chicago  0.000000
9    civics philanthropy  0.000000
11      institute armour  0.000000
1   architecture history  0.000000
8     chicago university  0.000000
5       building chicago  0.000000
4             art school  0.000000
18   university illinois  0.000000
['CPS' 'CS_Armour' 'CS_Sullivan' 'CS_Tallmadge' 'CS_civics']
19
5
19
1


In [3]:
#RandomForestClassifier - Ensemble Learning
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=10,criterion="gini", max_features=None, max_depth=None, \
                                min_samples_split=2, min_samples_leaf=1, max_leaf_nodes=None, \
                                class_weight="balanced", random_state=123)

rf_acc = cross_val_score(clf_rf,train,Y,scoring="accuracy",cv=6)
#print(acc)
print(np.mean(rf_acc))

clf_rf.fit(train, Y)

#print(len(clf_dt.feature_importances_))
#print(clf_dt.feature_importances_)
#print(clf_dt.tree_)
#tree.export_graphviz(clf_dt,out_file='tree.dot')
df = pd.DataFrame(data= {"Features" : tfidf.get_feature_names(), "weights" : clf_rf.feature_importances_})
df = df.sort_values(axis=0,by='weights',ascending=False)
print(df)

0.511051511281
                Features   weights
9    civics philanthropy  0.244480
16         school civics  0.169699
14   school architecture  0.147133
7         chicago school  0.137187
10        condit chicago  0.095494
17    university chicago  0.056201
3          art institute  0.049136
6            carl condit  0.040422
13              new york  0.019718
8     chicago university  0.015743
0   architecture chicago  0.008937
5       building chicago  0.005657
12  institute technology  0.005296
18   university illinois  0.002800
15        school chicago  0.002097
11      institute armour  0.000000
1   architecture history  0.000000
4             art school  0.000000
2       armour institute  0.000000


In [9]:
#Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression

#Cs = [0.001,0.01,0.1,1,10,100,1000]
Cs = [100]

#CW = ["balanced",None]
CW = ["balanced"]

#ITR = [50,100,200,300,400]
ITR = [100]

for c in Cs:
    for cw in CW:
        for itr in ITR:
            print("C : ",c,", class_weight : ",cw,", max_iter : ",itr)
            clf_lr = LogisticRegression(class_weight=cw,C=c,max_iter=itr,random_state=123)
            acc = cross_val_score(clf_lr,train,Y,scoring="accuracy",cv=cv)
            #print(acc)
            print(np.mean(acc))

clf_lr.fit(train,Y)

print(clf_lr.classes_)

print(len(clf_lr.coef_))

#print(clf_lr.coef_)
for i, cls in enumerate(clf_lr.classes_):
    print("\nFeature weights for class : ",cls,"\n")
    df = pd.DataFrame(data= {"Features" : tfidf.get_feature_names(), "weights" : clf_lr.coef_[i]})
    df = df.sort_values(axis=0,by='weights',ascending=False)
    print(df)
"""
y = np.array(Y)
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=1234)
preds = []
truths = []
for tr, tst in kfold.split(train):
    #print('%d training instances and %d testing instances' %(len(tr), len(test)))
    clf_lr_2 = LogisticRegression(class_weight="balanced",C=100,max_iter=100,random_state=123)
    clf_lr_2.fit(train[tr], y[tr])
    Y_pred = clf_lr_2.predict(train[tst])
    preds.extend(Y_pred)
    truths.extend(y[tst])
    acc = accuracy_score(y[tst], Y_pred)
    print(acc)
c_m = confusion_matrix(truths, preds,labels=clf_lr_2.classes_)
cm_df = pd.DataFrame(c_m,index=clf_lr.classes_,columns=clf_lr_2.classes_)
print(cm_df)
#print(confusion_matrix(y[tst], Y_pred, clf_lr.classes_))
"""

C :  100 , class_weight :  balanced , max_iter :  100
0.646268503002
['CPS' 'CS_Armour' 'CS_Sullivan' 'CS_Tallmadge' 'CS_civics']
5

Feature weights for class :  CPS 

                Features   weights
17    university chicago  6.534951
7         chicago school -1.226555
11      institute armour -2.956993
4             art school -3.354082
18   university illinois -4.130375
1   architecture history -4.227729
12  institute technology -4.404609
15        school chicago -4.626861
5       building chicago -4.765111
2       armour institute -4.907169
9    civics philanthropy -5.014145
16         school civics -5.096588
6            carl condit -5.562290
13              new york -5.601089
3          art institute -5.701100
10        condit chicago -5.936499
0   architecture chicago -6.455351
8     chicago university -8.618173
14   school architecture -9.948146

Feature weights for class :  CS_Armour 

                Features    weights
3          art institute  11.095114
18   university il

'\ny = np.array(Y)\nfrom sklearn.model_selection import KFold\nkfold = KFold(n_splits=5, shuffle=True, random_state=1234)\npreds = []\ntruths = []\nfor tr, tst in kfold.split(train):\n    #print(\'%d training instances and %d testing instances\' %(len(tr), len(test)))\n    clf_lr_2 = LogisticRegression(class_weight="balanced",C=100,max_iter=100,random_state=123)\n    clf_lr_2.fit(train[tr], y[tr])\n    Y_pred = clf_lr_2.predict(train[tst])\n    preds.extend(Y_pred)\n    truths.extend(y[tst])\n    acc = accuracy_score(y[tst], Y_pred)\n    print(acc)\nc_m = confusion_matrix(truths, preds,labels=clf_lr_2.classes_)\ncm_df = pd.DataFrame(c_m,index=clf_lr.classes_,columns=clf_lr_2.classes_)\nprint(cm_df)\n#print(confusion_matrix(y[tst], Y_pred, clf_lr.classes_))\n'

In [5]:
#Support Vector Machine Classifier
from sklearn.svm import SVC,LinearSVC

clf_svc = SVC(C=10,random_state=123,class_weight="balanced")
acc = cross_val_score(clf_svc,train,Y,scoring="accuracy",cv=cv)
print(acc)
print(np.mean(acc))

clf_svc.fit(train,Y)

for c in Cs:
    for cw in CW:
        for itr in ITR:
            print("C : ",c,", class_weight : ",cw,", max_iter : ",itr)
            clf_linear_svc = LinearSVC(C=c,random_state=123,class_weight=cw,max_iter=itr)
            acc = cross_val_score(clf_linear_svc,train,Y,scoring="accuracy",cv=cv)
            #print(acc)
            print(np.mean(acc))

clf_linear_svc.fit(train,Y)

for i, cls in enumerate(clf_linear_svc.classes_):
    print("\nFeature weights for class : ",cls,"\n")
    df = pd.DataFrame(data= {"Features" : tfidf.get_feature_names(), "weights" : clf_linear_svc.coef_[i]})
    df = df.sort_values(axis=0,by='weights',ascending=False)
    print(df)

[ 0.54285714  0.36363636  0.54545455  0.5625      0.58064516]
0.519018642648
C :  100 , class_weight :  balanced , max_iter :  100
0.571598415026

Feature weights for class :  CPS 

                Features   weights
17    university chicago  2.079219
11      institute armour -0.659891
7         chicago school -0.880844
13              new york -1.166040
4             art school -1.172603
18   university illinois -1.174168
2       armour institute -1.175396
12  institute technology -1.201965
15        school chicago -1.232092
5       building chicago -1.242903
9    civics philanthropy -1.335171
16         school civics -1.426332
3          art institute -1.469628
1   architecture history -1.516180
10        condit chicago -1.529153
6            carl condit -1.560202
0   architecture chicago -1.838295
14   school architecture -2.209645
8     chicago university -4.367333

Feature weights for class :  CS_Armour 

                Features       weights
3          art institute  5.134192e+0

In [6]:
#Neural Network Classifier
from sklearn.neural_network import MLPClassifier
"""
act = ["logistic","tanh","relu"]
hl = [(5,5),(5,10),(10,10),(10,15),(15,15),(15,20),(20,20)]
lr = ["constant","adaptive"]
erly_stop = [True,False]
shuffle = [True,False]
solver = ["lbfgs","sgd"]
alpha = [0.01,0.1,1,10,100]

for a in act:
    for h in hl:
        for l in lr:
            for e_s in erly_stop:
                for s in shuffle:
                    for m in ITR:
                        for so in solver:
                            for al in alpha:
                                print("Activation : ",a," hidden layers : ",h," learning rate : ",l," early_stopping : ",e_s," Shuffle : ",s," max_iter : ",m," solver :",so," Alpha : ",a)
                                clf_NN = MLPClassifier(hidden_layer_sizes=h,activation=a,solver=so,alpha=al,learning_rate=l,max_iter=m,shuffle=s,early_stopping=e_s,random_state=123)
                                acc = cross_val_score(clf_NN,train,Y,scoring="accuracy",cv=cv)
                                #print(acc)
                                print(np.mean(acc))

"""

clf_NN = MLPClassifier(hidden_layer_sizes=(5,5), activation="logistic", solver="lbfgs", alpha=0.1, \
                       learning_rate="constant", max_iter=300, shuffle=False, early_stopping=True, \
                       random_state=123)
NN_acc = cross_val_score(clf_NN,train,Y,scoring="accuracy",cv=cv)
#print(acc)
print(np.mean(NN_acc))

clf_NN.fit(train,Y)
print(len(clf_NN.coefs_))
#print(clf_NN.coefs_)
print(clf_NN.classes_)

"""
for i, cls in enumerate(clf_NN.classes_):
    print("\nFeature weights for class : ",cls,"\n")
    df = pd.DataFrame(data= {"Features" : tfidf.get_feature_names(), "weights" : clf_NN.coefs_[i]})
    df = df.sort_values(axis=0,by='weights',ascending=False)
    print(df)
"""
"""
for i, layer_weights in enumerate(clf_NN.coefs_):
    print("\nLayer : ",i)
    for j, feature in enumerate(tfidf.get_feature_names()):
        print("\nFeature : ",feature)
        print("\nCorresponding weights in layer(Note : The ith element in the list represents the weight matrix corresponding to layer i.) : ")
        print(layer_weights[j])
"""

for j, feature in enumerate(tfidf.get_feature_names()):
    print("\nFeature : ",feature)  
    print(clf_NN.coefs_[0][j])
    """
    for i, layer_weights in enumerate(clf_NN.coefs_[0]):
        print("Layer : ",i)
        #print("\nCorresponding weights in layer(Note : The ith element in the list represents the weight matrix corresponding to layer i.) : ")
        print(layer_weights[j])
    """

0.628433005167
3
['CPS' 'CS_Armour' 'CS_Sullivan' 'CS_Tallmadge' 'CS_civics']

Feature :  architecture chicago
[-1.29796836  1.51467781 -1.19903216  0.14965242  0.42861058]

Feature :  architecture history
[ 0.63810571 -0.39548631 -1.18199277  0.48327389  0.44729108]

Feature :  armour institute
[-1.33798021  0.93776699  0.11990895  0.60719221 -0.56078082]

Feature :  art institute
[-2.0542376   1.13760072  0.44246113  1.23982586 -1.21849858]

Feature :  art school
[-1.09063163  0.72822279  0.14753915  0.48426503 -0.47093018]

Feature :  building chicago
[-0.21778443  1.10154319 -1.07826517 -0.63447455  1.14366181]

Feature :  carl condit
[ 1.45798126 -1.34504964 -1.88755399  1.08289292  0.59574408]

Feature :  chicago school
[ 1.03279949  0.5005507  -0.13855228 -2.1172376   2.12872141]

Feature :  chicago university
[-0.81430997  0.57287546 -2.39558001  1.5289156   0.16402087]

Feature :  civics philanthropy
[-0.039446   -1.3138943   0.80130807  1.91005309 -1.74163358]

Feature :  con

In [10]:
testData = []
#list of all snippet files in the test data
file_list = [f for f in listdir(testSnippetFolder) if not f.startswith('.')]

for file in file_list:
    filePath = testSnippetFolder + "/" + file
    f = open(filePath,'r',encoding='utf-8')
    data = f.readlines()
    textSnippet = ''.join(data)
    textSnippet = textSnippet.replace("\n","")
    for p in puncs:
        if p in textSnippet:
            textSnippet = textSnippet.replace(p,"")
    testData.append(textSnippet)
    
#use same vectorizer used to train data
test = tfidf.transform(testData)


In [19]:
#which classifier to use for classification of the test data?
Y_test_lr = clf_lr.predict(test)
probs = clf_lr.predict_proba(test)
print(clf_lr.classes_)
print(probs)

Y_probs = []

classes = list(clf_lr.classes_)

for i,label in enumerate(Y_test_lr):
    label_index = classes.index(label)
    max_prob = probs[i][label_index]
    Y_probs.append(max_prob)
    
label_prob = []
l = len(Y_test_lr)
for i in range(l):
    label_prob.append((file_list[i],Y_test_lr[i],Y_probs[i]))
print(label_prob)
import pickle

pickle.dump(label_prob, open(folderToSaveResultsFile + "/" + 'Snippets.pkl', 'wb'))

"""
Y_test_svc = clf_svc.predict(test)

Y_test_linear_svc = clf_linear_svc.predict(test)

Y_test_nn = clf_NN.predict(test)
"""
"""

print(len(file_list))
print(len(Y_test_lr))

#final list containing of (snippet file name, predicted class label) 
final_list = []
l = len(file_list)
for i in range(l):
    final_list.append((file_list[i],Y_test_lr[i]))
print(final_list)

#dict of string(key) : list (value)
#class label as key, and list of snippets classified into this label as values
d = dict()
for t in final_list:
    if t[1] not in d.keys():
        d[t[1]] = [t[0]]
    else:
        d[t[1]].append(t[0])

for key, values in d.items():
    with open(folderToSaveResultsFile + "/" + key + ".txt","a") as out_file:
        for item in values :
            out_file.write(item+"\n")

#print(d)
print("Done")
"""
print(clf_lr.classes_)
print(Y_test_lr)
print(probs)


['CPS' 'CS_Armour' 'CS_Sullivan' 'CS_Tallmadge' 'CS_civics']
[[  3.73167515e-03   4.35870770e-01   2.82450320e-01   2.77258020e-01
    6.89214677e-04]
 [  3.73167515e-03   4.35870770e-01   2.82450320e-01   2.77258020e-01
    6.89214677e-04]
 [  3.58619548e-01   3.87094945e-02   1.85895710e-01   4.16191780e-01
    5.83467808e-04]
 [  3.73167515e-03   4.35870770e-01   2.82450320e-01   2.77258020e-01
    6.89214677e-04]
 [  3.58619548e-01   3.87094945e-02   1.85895710e-01   4.16191780e-01
    5.83467808e-04]
 [  3.58619548e-01   3.87094945e-02   1.85895710e-01   4.16191780e-01
    5.83467808e-04]
 [  6.11416536e-03   3.51802651e-01   2.85995746e-01   3.53921355e-01
    2.16608202e-03]
 [  2.44855214e-02   1.22209126e-02   1.46813113e-02   7.17514091e-03
    9.41437114e-01]
 [  3.73167515e-03   4.35870770e-01   2.82450320e-01   2.77258020e-01
    6.89214677e-04]
 [  3.88285089e-01   1.99822294e-02   5.84697437e-01   2.42960929e-03
    4.60563533e-03]
 [  3.73167515e-03   4.35870770e-01   2

In [9]:
#try clustering techniques

In [17]:
#try word2vec model
from nltk.corpus import brown
from gensim.models import Word2Vec

w2v_model = Word2Vec(brown.sents(),size=50,window=5,min_count=1,workers=2)
w2v_model_wv = w2v_model.wv
del w2v_model

In [97]:
def make_feature_dicts(data, w2v_model_wv, w2v=True, token=True, caps=True, context=True):
    list_of_dicts = []
    #labels = []

    for sentence_data in data:
        #print(sentence_data)
        sentence_dict = dict()   
        words = sentence_data.split(" ")
        sentence_length = len(words)
        #print(sentence_length)
        current_token = ""
        current_is_caps = 0
        current_wv = []
        prev_wv = []
        next_wv = []
        prev_token = ""
        prev_is_caps = 0      
        next_token = ""
        next_is_caps = 0
        
        for i in range(sentence_length-1):
            #print(i,"th", " Word : ",words[i],"Length : ",len(words[i]))
            #print(i+1,"th", " Word : ",words[i+1],"Length : ",len(words[i+1]))
            if len(words[i]) <= 0:
                continue
            temp = dict()
            #labels.append(sentence_data[i][3])
            if context:
                if(sentence_length == 1):#only one word in the sentence
                    if token:
                        current_token = words[i].lower()
                        temp.update({"tok="+current_token:1})
                    if caps:
                        if len(words[i]) == 1:
                            if words[i].isupper():
                                temp.update({"is_caps":1})
                        else:
                            if words[i][0].isupper():
                                temp.update({"is_caps":1})
                    if w2v:
                        try:
                            current_wv = w2v_model_wv[words[i]]
                            for k in range(len(current_wv)):
                                temp.update({"w2v_"+ str(k) : current_wv[k]})
                        except Exception as e:
                            current_wv = []
                            sentence_dict.update(temp)
                            continue
                elif i > 0 and i+1 < sentence_length:#middle words
                    if token:
                        prev_token = current_token
                        current_token = next_token
                        next_token = words[i+1].lower()
                        temp.update({"tok="+current_token:1, "prev_tok="+prev_token:1, "next_tok="+next_token:1})
                    if caps:
                        prev_is_caps = current_is_caps
                        if prev_is_caps:
                            temp.update({"prev_is_caps":1})
                            current_is_caps = next_is_caps
                        if current_is_caps:
                            temp.update({"is_caps":1})
                            if len(words[i+1]) == 1:
                                if words[i+1].isupper():
                                    next_is_caps = 1
                                    temp.update({"is_caps":1})
                                else:
                                    next_is_caps = 0
                            else:
                                if len(words[i+1]) <= 0:
                                    next_is_caps = 0
                                    continue
                                if words[i+1][0].isupper():
                                    next_is_caps = 1
                                    temp.update({"next_is_caps":1})
                                else:
                                    next_is_caps = 0
                    if w2v:
                        prev_wv = current_wv
                        for k in range(len(prev_wv)):
                            temp.update({"prev_w2v_" + str(k) : prev_wv[k]})
                        current_wv = next_wv
                        for k in range(len(current_wv)):
                            temp.update({"w2v_" + str(k) : current_wv[k]})
                        try:
                            next_wv = w2v_model_wv[words[i+1]]
                            for k in range(len(next_wv)):
                                temp.update({"next_w2v_" + str(k) : next_wv[k]})
                        except Exception as e:
                            next_wv = []
                            sentence_dict.update(temp)
                            continue
                elif i+1 == sentence_length:#last but one token
                    if token:
                        prev_token = current_token
                        current_token = next_token
                        temp.update({"tok="+current_token:1, "prev_tok="+prev_token:1})
                    if caps:
                        prev_is_caps = current_is_caps
                        if prev_is_caps:
                            temp.update({"prev_is_caps":1})
                            current_is_caps = next_is_caps
                        if current_is_caps:
                            temp.update({"is_caps":1})
                    if w2v:
                        prev_wv = current_wv
                        current_wv = next_wv
                        for k in range(len(prev_wv)):
                            temp.update({"prev_w2v_" + str(k) : prev_wv[k]})
                        for k in range(len(current_wv)):
                            temp.update({"w2v_" + str(k) : current_wv[k]})
                else:#first token
                    if token:
                        current_token = words[i].lower()
                        next_token = words[i+1].lower()
                        temp.update({"tok="+current_token:1, "next_tok="+next_token:1})
                    if caps:
                        if words[i][0].isupper():
                            current_is_caps = 1
                            temp.update({"is_caps":1})
                        else:
                            current_is_caps = 0
                            if len(words[i+1]) == 1:
                                if words[i+1].isupper():
                                    next_is_caps = 1
                                    temp.update({"next_is_caps":1})
                                else:
                                    next_is_caps = 0
                            else:
                                if len(words[i+1]) <= 0:
                                    next_is_caps = 0
                                    continue
                                if words[i+1][0].isupper():
                                    next_is_caps = 1
                                    temp.update({"next_is_caps":1})
                                else:
                                    next_is_caps = 0
                    if w2v:
                        try:
                            current_wv = w2v_model_wv[words[i]]
                            for k in range(len(current_wv)):
                                temp.update({"w2v_" + str(k) : current_wv[k]})
                        except Exception as e:
                            current_wv = []
                            #continue
                        try:
                            next_wv = w2v_model_wv[words[i+1]]
                            for k in range(len(current_wv)):
                                temp.update({"next_w2v_" + str(k) : next_wv[k]})
                        except Exception as e:
                            next_wv = []
                            sentence_dict.update(temp)
                            continue
            else:
                if token:
                    current_token = words[i].lower()
                    temp.update({"tok="+current_token:1})
                if caps:
                    if len(words[i]) == 1:
                        if words[i].isupper():
                            temp.update({"is_caps":1})
                    else:
                        if words[i][0].isupper():
                             temp.update({"is_caps":1})
                if w2v:
                    try:
                        current_wv = w2v_model_wv[words[i]]
                        for k in range(len(current_wv)):
                            temp.update({"w2v_" + str(k) : current_wv[k]})
                    except Exception as e:
                        current_wv = []
                        sentence_dict.update(temp)
                        continue
            sentence_dict.update(temp)
        #print(temp)
        #print()
        list_of_dicts.append(sentence_dict)
    return list_of_dicts

In [104]:
#print(w2v_model_wv["Chicago"])
#print("Something")
from sklearn.feature_extraction import DictVectorizer
train_dicts = make_feature_dicts(X, w2v_model_wv, w2v=True, token=True, caps=False, context=True)
vec = DictVectorizer()
X_train_v = vec.fit_transform(train_dicts)

print(X_train_v.shape)

clf_linear_svc = LinearSVC(C=c, random_state=123, class_weight=cw, max_iter=itr)
acc = cross_val_score(clf_linear_svc, X_train_v, Y,scoring="accuracy", cv=cv)
print(np.mean(acc))

clf_linear_svc.fit(X_train_v, Y)

(164, 6105)
0.682986489317


LinearSVC(C=100, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=100,
     multi_class='ovr', penalty='l2', random_state=123, tol=0.0001,
     verbose=0)

In [99]:
test_dicts = make_feature_dicts(testData, w2v_model_wv, w2v=True, token=True, caps=True, context=True)
X_test_v = vec.transform(test_dicts)

Y_pred = clf_linear_svc.predict(X_test_v)

print("Done")

Done
